In [1]:
import pandas as pd
import numpy as np

In [2]:
df_film = pd.read_csv("df_film.csv")

## Ajout données de tmdb

Pour ajouter des données telles que la popularité, les pays de production, les affiches de films, nous avons du nous servir du tmdb.csv

In [ ]:
df_tmbd = pd.read_csv("tmdb_full.csv")

In [ ]:
df_tmbd = df_tmbd.drop(columns = [
    "adult",
    "backdrop_path",
    "budget",
    "genres",
    "homepage",
    "id",
    "original_language",
    "original_title",
    "overview",
    "release_date",
    "revenue",
    "runtime",
    "spoken_languages",
    "tagline",
    "title",
    "video",
    "vote_average",
    "vote_count",
    "production_companies_name",
    "production_companies_country"])

Merge nos nouvelles données avec le df_film 

In [ ]:
df_film = pd.read_csv("df_film.csv")
df_merge = df_film.merge(df_tmbd, left_on = "id_film", right_on = "imdb_id", how = "left")
df_merge = df_merge.drop(columns = ["Unnamed: 0.2","Unnamed: 0.1", "Unnamed: 0", "tconst"])

## Créer des colonnes supplémentaires

Créer la colonne décénie

In [3]:
df_film["decenie"] = ((df_film['année'] // 10) * 10).astype(str)

## Remplacer des valeurs manquantes

Transformer les valeurs nulles en format compréhensible par python. Dans notre df actuellement les valeurs nulles sont des strings '\\N'

In [4]:
df_film = df_film.replace('\\N', np.nan)

### Durée (temps_minutes)

Ici, on veut remplacer les valeurs nulles par la moyenne des autres films de la même décénie

Converti le temps (str) en int en contournant les valeurs nulles (fonctionne uniquement avec un type compris par python)

In [20]:
df_film['temps_minutes'] = df_film['temps_minutes'].astype('Int64')

Cacul de la moyenne par décénie

In [21]:
moy_decenie = df_film.groupby("decenie")["temps_minutes"].mean()

fonction qui remplace les valeurs nulles par la moyenne de la décénie à laquelle il appartient

In [22]:
def replacena_mean(row):
    if str(row["temps_minutes"]) == "<NA>":
        row["temps_minutes"] = moy_decenie.loc[row["decenie"]]
    return row

In [23]:
# Appliquer cette fonction
df_film = df_film.apply(replacena_mean, axis=1)

In [24]:
df_film.to_csv("df_film.csv")

### Années

In [ ]:
# Création d'une liste de films qui ont pour année 0
films_annee0 = df_film.loc[df_film["année"] == 0, "titre"].tolist()

In [ ]:
# Dictionnaire avec les années manquantes au DF
films = {
    "Shubh Kaamna": 1983,
    "You Don't Know Nicotine": 2020,
    "Becoming Led Zeppelin": 2021,
    "Concorde, le rêve supersonique": 2020,
    "Luccas Neto em: Acampamento de Férias": 2019,
    "Luccas Neto em: Acampamento de Férias 2": 2020,
    "Prem Prakaran": 2021,
    "El Arribo de Conrado Sierra": 2012,
    "Cyborg Nemesis: The Dark Rift" : 2014,
    "90° South" : 1933,
    "Samhain" : 2021,
    "Haunted Connecticut" : 2009,
}

# Apply du dictionnaire pour remplacer les valeurs manquantes
df_film["année"] = df_film.apply(lambda row: films.get(row["titre"], row["année"]), axis=1)

In [ ]:
# Supression des lignes restantes avec année non renseignée
df_film= df_film[df_film["année"] != 0]

## Genre les + vus par décénie

La colonne genre contient une string avec plusieurs genres différents. Je veux dans un premier temps transformer ces chaînes de caractères en listes.

Pour éviter les bugs avec split, je remplace les valeurs manquantes par des liste vides.

In [5]:
df_film['genre'] = df_film['genre'].fillna("")

Je créé donc une fonction qui va prendre la chaîne de caractère en paramètre et renvoyer la liste pour chaque ligne

In [6]:
def transfo_liste(chaine:str)-> list:
    liste_genres = []
    for genre in chaine.split(","):
        liste_genres.append(genre.strip())
    return liste_genres


Je l'applique sur la colonne que je veux. Ici, je transforme la colonne genre (initialement des strings) en une nouvelle colonne genre qui contient maintenant des listes de genre.

In [7]:
df_film['genre'] = df_film['genre'].apply(transfo_liste)

Maintenant, je voudrais compter les nombres de films de chaque genres par décénie. Pour ça je veux faire un dictionnaire qui à en clé la décénie, et en valeur, les nombres de films par genre [pour cette même décénie donc].

In [8]:
resultats_par_decennie = {}

for decenie in df_film["decenie"].unique():
    df_dec = df_film.loc[df_film["decenie"] == decenie, ['genre', 'decenie']].explode("genre")
    nb_genres = df_dec.groupby('genre')["genre"].count()
    resultats_par_decennie[decenie] = nb_genres

Je voudrais transformer ce dictionnaire en dataframe pour ensuite faire des visualisation graphiques

In [9]:
df_genre_decenie = pd.DataFrame(resultats_par_decennie).fillna(0)

### Objectif: heatmap

L'objectif est de réaliser une heatmap (dans la partie analyse). Une heatmap qui prend en axe horizontal la décénie et en axe vertical les 10 genres les plus populaires. La couleur ne doit pas représenter le nombre de films brut mais plutôt le ratio que chaque genre représente par décénie. 

1) On commence à travailler sur tout les genres (pas le top 10 pour l'instant)

Pour calculer le ration il faut faire = case / somme_colonne, on prendra les cases du dataframe top 10 qu'on divisera par la somme par colonne ce df plus global. On calcul donc la somme par colonne (par décénie)

In [10]:
sum_par_dec = df_genre_decenie.sum(axis = 0) #calcul de la somme_colonne

2. Pour le top 10, je fais une somme des genres que je trie par ordre croissant et desquels je ne garde que le top 10 (head10)

In [11]:
df_genre_decenie["sum_genre"] = df_genre_decenie.sum(axis = 1)
df_top_genre = df_genre_decenie.sort_values("sum_genre", ascending = False).head(10)

Je supprimme ensuite la colonne sum_genre qui ne servait qu'à créer ce nouveau dataframe

In [12]:
df_top_genre = df_top_genre.drop(columns = 'sum_genre')

Maintenant je fais le ratio avec la somme par colonne du dataframe total (sans le filtre du top10)

In [14]:
df_top_genre = df_top_genre / sum_par_dec

In [18]:
df_top_genre.to_csv("df_top_genre.csv")